In [ ]:
from google.cloud import secretmanager

from faunadb import query as q
from faunadb.objects import Ref
from faunadb.client import FaunaClient

def get_client():
    print('grabbing secret')

    client = secretmanager.SecretManagerServiceClient()
    secret_name = "fauna_deepcite_db"
    project_id = "deepcite-306405"

    request = {"name": f"projects/{project_id}/secrets/{secret_name}/versions/latest"}
    response = client.access_secret_version(request)
    secret_string = response.payload.data.decode("UTF-8")

    return FaunaClient(secret=secret_string, domain='db.us.fauna.com')

client = get_client()

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 100
import json

def fauna_to_df(fauna):
  data = [doc['data'] for doc in fauna['data']]
  df = pd.DataFrame.from_records(data)

  return df

def grab_fauna(collection_name, size = 100000):
  return client.query(
    q.map_(
      lambda x: q.get(x),
      q.paginate(q.documents(q.collection(collection_name)), size=size)
    )
  )



In [ ]:
import psycopg2
import pandas as pd
pd.options.display.max_colwidth = 100
import json

def grab_postgres(table):
    with open('database_config.json') as json_file:
        db_config = json.load(json_file)['gcp']
    conn = psycopg2.connect(host=db_config['host'], user=db_config['user'], password=db_config['password'], database=db_config['database'], port=db_config['port'])
    sql = f'SELECT * FROM "{table}" ORDER by "created_at" DESC;'
    call_df_postgres = pd.read_sql_query(sql, conn)
    return call_df_postgres


In [ ]:
def grab_data_and_compare(collection, table):
    df_postgres = grab_postgres(table).drop_duplicates(subset=['base_id', 'source_id', 'user_id', 'stage', 'redact'])
    fauna = grab_fauna(collection)
    df_fauna = fauna_to_df(fauna).drop_duplicates(subset=['base_id', 'source_id', 'user_id', 'stage', 'redact'])

    df = pd.merge(df_postgres, df_fauna, on=['base_id', 'source_id', 'user_id', 'stage', 'redact'], how='left', indicator='Exist')
    df = df[['user_id', 'base_id', 'source_id', 'stage', 'current_versions_x',
       'created_at', 'redact', 'Exist']]
    df = df.rename(columns={'current_versions_x': 'current_versions'})
    # df['Exist'] = np.where(df.Exist == 'both', True, False)

    # missing_df_postgres = df_postgres[~df_postgres['id'].isin(df_fauna['id'])]
    missing_df_postgres = df[df['Exist'] == 'left_only'].drop(columns=['Exist'])
    print(f'length of missing postgres: {len(missing_df_postgres)}')

    same_df_postgres = df[df['Exist'] == 'both'].drop(columns=['Exist'])
    print(f'length of matching postgres and fauna: {len(same_df_postgres)}')

    # call_df_fauna_which_match_postgres = df_fauna[df_fauna['id'].isin(same_df_postgres['id'])] ## Remove the two rows only present in fauna
    print(f'''Check if length of postges match fauna: 
    {len(df_fauna) == len(same_df_postgres)}''')

    return same_df_postgres, fauna, missing_df_postgres

same_df_postgres, fauna, missing_df_postgres = grab_data_and_compare('deepcite_source', 'source_label')



In [ ]:
len(missing_df_postgres)

## Prepare missing json data for fauna

In [ ]:
# Correct postgres after merge
# same_df_postgres_clean = same_df_postgres[['user_id', 'base_id', 'source_id', 'stage', 'current_versions_x',
#        'created_at', 'redact']]
# same_df_postgres_clean = same_df_postgres_clean.rename(columns={'current_versions_x': 'current_versions'})


# Drop duplicate json data
fauna_json_clean = []
fauna_json = [doc['data'] for doc in fauna['data']]
for value in fauna_json:
    if not [item for item in fauna_json_clean if check_json_matches(value,item)]:
        fauna_json_clean.append(value)

In [ ]:
def ordered(obj):
    if isinstance(obj, dict):
        return sorted((k, ordered(v)) for k, v in obj.items())
    if isinstance(obj, list):
        return sorted(ordered(x) for x in obj)
    else:
        return obj

def check_json_matches(a,b):
    return ordered(a) == ordered(b)

same_json = json.loads(same_df_postgres.to_json(orient='records'))
for obj in same_json:
    obj.pop('created_at')

len(same_json) - len(fauna_json_clean)
print(f"Does the json from postgres match json from fauna: {check_json_matches(same_json, fauna_json_clean)}")



In [ ]:
json_missing = json.loads(missing_df_postgres.to_json(orient='records'))
with open('missing_deepcite_source.json', 'w', encoding='utf-8') as f:
    json.dump(json_missing, f, ensure_ascii=False, indent=4)

In [ ]:
# ! brew install fauna-shell

In [ ]:
! fauna import --path=./missing_deepcite_source.json --collection=deepcite_call --append --dry-run